# VacationPy
--In the 2nd part of the data analysis (VacationPy) we use the weather information we gathered to determine a good vacation spot. 

Disclaimer: For some reason, my code didnt find any hotels in any of the locations and I got several tutors to look at my code.-

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import hvplot.pandas
import geopandas as gpd
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,geraldton,-28.7667,114.6000,72.01,78,0,17.27,AU,clear sky
1,1,nanortalik,60.1432,-45.2371,26.04,83,100,35.79,GL,overcast clouds
2,2,nyurba,63.2842,118.3319,-49.81,100,62,3.83,RU,broken clouds
3,3,mar del plata,-38.0023,-57.5575,73.42,91,40,9.22,AR,scattered clouds
4,4,port lincoln,-34.7333,135.8667,64.51,93,100,8.28,AU,overcast clouds


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map_weather = city_data_df.hvplot.points("Lng","Lat", size="Humidity",color="City", geo=True, frame_width=700, frame_height=500, tiles="OSM", scale=1, alpha=.5)
#hvplot.pandas.holoviews.ipython.show_traceback()
# Display the map
map_weather

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
good_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 72)
                                    & (city_data_df["Max Temp"] > 55)  
                                    & (city_data_df["Wind Speed"] < 10)
                                    & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
good_weather_df = good_weather_df.dropna()

# Display sample data
good_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,40,sabha,27.0377,14.4283,61.02,18,0,6.73,LY,clear sky
150,150,coquimbo,-29.9533,-71.3436,65.93,77,0,2.30,CL,clear sky
202,202,san carlos de bariloche,-41.1456,-71.3082,62.35,36,0,5.75,AR,clear sky
238,238,pasan,22.8500,82.2000,63.54,50,0,3.78,IN,clear sky
288,288,wanaka,-44.7000,169.1500,57.11,72,0,1.99,NZ,clear sky
296,296,hawalli,29.2833,47.9972,66.29,42,0,3.00,KW,clear sky
316,316,lata,40.1629,-8.3327,56.07,40,0,9.22,PT,clear sky
322,322,madhoganj,27.1167,80.1500,57.61,41,0,6.17,IN,clear sky
348,348,taunggyi,20.7833,97.0333,56.53,53,0,1.88,MM,clear sky
352,352,buraydah,26.3260,43.9750,66.65,40,0,6.69,SA,clear sky


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = good_weather_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Hotel Name
40,sabha,LY,27.0377,14.4283,
150,coquimbo,CL,-29.9533,-71.3436,
202,san carlos de bariloche,AR,-41.1456,-71.3082,
238,pasan,IN,22.8500,82.2000,
288,wanaka,NZ,-44.7000,169.1500,
296,hawalli,KW,29.2833,47.9972,
316,lata,PT,40.1629,-8.3327,
322,madhoganj,IN,27.1167,80.1500,
348,taunggyi,MM,20.7833,97.0333,
352,buraydah,SA,26.3260,43.9750,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address1 = name_address.json()
    print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address1["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
         #Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
<Response [200]>
sabha - nearest hotel: No hotel found
<Response [200]>
coquimbo - nearest hotel: No hotel found
<Response [200]>
san carlos de bariloche - nearest hotel: No hotel found
<Response [200]>
pasan - nearest hotel: No hotel found
<Response [200]>
wanaka - nearest hotel: No hotel found
<Response [200]>
hawalli - nearest hotel: No hotel found
<Response [200]>
lata - nearest hotel: No hotel found
<Response [200]>
madhoganj - nearest hotel: No hotel found
<Response [200]>
taunggyi - nearest hotel: No hotel found
<Response [200]>
buraydah - nearest hotel: No hotel found
<Response [200]>
tura - nearest hotel: No hotel found
<Response [200]>
along - nearest hotel: No hotel found
<Response [200]>
yanam - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
40,sabha,LY,27.0377,14.4283,No hotel found
150,coquimbo,CL,-29.9533,-71.3436,No hotel found
202,san carlos de bariloche,AR,-41.1456,-71.3082,No hotel found
238,pasan,IN,22.8500,82.2000,No hotel found
288,wanaka,NZ,-44.7000,169.1500,No hotel found
296,hawalli,KW,29.2833,47.9972,No hotel found
316,lata,PT,40.1629,-8.3327,No hotel found
322,madhoganj,IN,27.1167,80.1500,No hotel found
348,taunggyi,MM,20.7833,97.0333,No hotel found
352,buraydah,SA,26.3260,43.9750,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot

# YOUR CODE HERE
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    hover_cols = 'all',
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,index,Country,Hotel Name)